🔴 Avancé | ⏱ 45 min | 🔑 Concepts : logging, JSON logging, Datadog/Splunk ready

# 06 - Logging Structuré et Production-Ready

## Objectifs

- Comprendre pourquoi `print()` n'est pas suffisant
- Maîtriser le module **logging** de Python
- Configurer des **niveaux** et **handlers** appropriés
- Implémenter du **JSON logging** pour les outils modernes (Datadog, Splunk, ELK)
- Appliquer les bonnes pratiques de logging en production

## Prérequis

- Python 3.8+
- Compréhension des concepts de base (fonctions, classes)
- Familiarité avec JSON (optionnel)

## 1. print() vs logging : Pourquoi logger ?

### Limites de print()

```python
# ❌ Problèmes avec print()
print("Starting process...")           # Pas de timestamp
print("User:", user_id)                # Pas de niveau de gravité
print("ERROR: Database connection")    # Mélangé avec stdout
```

### Avantages du logging

| Fonctionnalité | print() | logging |
|----------------|---------|----------|
| **Niveaux** (DEBUG, INFO, ERROR) | ❌ | ✅ |
| **Timestamp automatique** | ❌ | ✅ |
| **Filtrage** par niveau | ❌ | ✅ |
| **Multiples destinations** (fichier, console, API) | ❌ | ✅ |
| **Rotation de logs** | ❌ | ✅ |
| **Contexte** (module, ligne, fonction) | ❌ | ✅ |
| **Production-ready** | ❌ | ✅ |

## 2. Module logging : Les bases

In [ ]:
import logging

# Configuration basique
logging.basicConfig(level=logging.INFO)

# Utilisation
logging.debug("Message de debug (ne s'affiche pas car level=INFO)")
logging.info("Information importante")
logging.warning("Attention !")
logging.error("Une erreur s'est produite")
logging.critical("Erreur critique !")

## 3. Les 5 niveaux de logging

| Niveau | Valeur | Quand l'utiliser ? | Exemple |
|--------|--------|-------------------|----------|
| **DEBUG** | 10 | Informations de débogage détaillées | "Variable x = 42", "Entrée dans fonction()" |
| **INFO** | 20 | Confirmation que tout fonctionne | "Serveur démarré sur port 8000" |
| **WARNING** | 30 | Quelque chose d'inattendu, mais pas grave | "Disque presque plein (80%)" |
| **ERROR** | 40 | Erreur grave, mais le programme continue | "Échec de connexion à la DB" |
| **CRITICAL** | 50 | Erreur qui arrête le programme | "Mémoire épuisée, arrêt imminent" |

In [ ]:
import logging

# Configurer pour afficher tous les niveaux
logging.basicConfig(
    level=logging.DEBUG,
    format='%(levelname)-8s - %(message)s'
)

logging.debug("Détails pour le débogage")
logging.info("Info générale")
logging.warning("Avertissement")
logging.error("Erreur")
logging.critical("Critique !")

## 4. Formatage des logs

In [ ]:
import logging

# Format personnalisé
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s | %(levelname)-8s | %(name)s | %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

logger = logging.getLogger(__name__)
logger.info("Message avec format personnalisé")
logger.error("Une erreur avec timestamp")

### Attributs de formatage disponibles

| Attribut | Description | Exemple |
|----------|-------------|----------|
| `%(asctime)s` | Timestamp | `2024-02-08 14:30:00` |
| `%(levelname)s` | Niveau | `INFO`, `ERROR` |
| `%(name)s` | Nom du logger | `__main__`, `myapp.module` |
| `%(message)s` | Message | `User logged in` |
| `%(funcName)s` | Nom de la fonction | `process_data` |
| `%(lineno)d` | Numéro de ligne | `42` |
| `%(pathname)s` | Chemin complet du fichier | `/app/main.py` |
| `%(filename)s` | Nom du fichier | `main.py` |
| `%(process)d` | ID du processus | `12345` |
| `%(thread)d` | ID du thread | `67890` |

## 5. Loggers par module : Bonne pratique

In [ ]:
%%writefile user_service.py
"""Service utilisateur."""
import logging

# ✅ BON : Logger par module
logger = logging.getLogger(__name__)

def create_user(username):
    """Crée un utilisateur."""
    logger.info(f"Creating user: {username}")
    # ...
    logger.debug(f"User {username} created successfully")

def delete_user(user_id):
    """Supprime un utilisateur."""
    logger.warning(f"Deleting user {user_id}")
    # ...

In [ ]:
%%writefile database.py
"""Module database."""
import logging

logger = logging.getLogger(__name__)

def connect():
    """Connexion à la DB."""
    logger.info("Connecting to database...")
    # ...
    logger.info("Database connected")

def query(sql):
    """Exécute une requête."""
    logger.debug(f"Executing query: {sql}")
    # ...

In [ ]:
import logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s | %(levelname)-8s | %(name)-20s | %(message)s'
)

# Importer les modules
import user_service
import database

# Utiliser les fonctions
database.connect()
user_service.create_user("alice")
database.query("SELECT * FROM users")

## 6. Handlers : Destinations multiples

In [ ]:
import logging

# Créer un logger
logger = logging.getLogger('my_app')
logger.setLevel(logging.DEBUG)

# Handler 1 : Console (INFO et au-dessus)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
console_format = logging.Formatter('%(levelname)s - %(message)s')
console_handler.setFormatter(console_format)

# Handler 2 : Fichier (DEBUG et au-dessus)
file_handler = logging.FileHandler('app.log')
file_handler.setLevel(logging.DEBUG)
file_format = logging.Formatter('%(asctime)s | %(levelname)s | %(message)s')
file_handler.setFormatter(file_format)

# Handler 3 : Fichier d'erreurs (ERROR et au-dessus)
error_handler = logging.FileHandler('errors.log')
error_handler.setLevel(logging.ERROR)
error_handler.setFormatter(file_format)

# Ajouter les handlers
logger.addHandler(console_handler)
logger.addHandler(file_handler)
logger.addHandler(error_handler)

# Tester
logger.debug("Debug message (seulement dans app.log)")
logger.info("Info message (console + app.log)")
logger.error("Error message (console + app.log + errors.log)")

In [ ]:
# Vérifier le contenu des fichiers
!echo "=== app.log ==="
!cat app.log
!echo "\n=== errors.log ==="
!cat errors.log

### Types de handlers courants

| Handler | Usage |
|---------|-------|
| `StreamHandler` | Console (stdout/stderr) |
| `FileHandler` | Fichier simple |
| `RotatingFileHandler` | Fichier avec rotation par taille |
| `TimedRotatingFileHandler` | Fichier avec rotation par temps |
| `SocketHandler` | Envoi via socket réseau |
| `HTTPHandler` | Envoi via HTTP |
| `SMTPHandler` | Envoi par email |
| `SysLogHandler` | Syslog système |

## 7. Rotation de logs

In [ ]:
import logging
from logging.handlers import RotatingFileHandler, TimedRotatingFileHandler

logger = logging.getLogger('rotating_app')
logger.setLevel(logging.DEBUG)

# Rotation par taille : max 1MB, garder 5 backups
size_handler = RotatingFileHandler(
    'app_size.log',
    maxBytes=1024 * 1024,  # 1 MB
    backupCount=5
)
size_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
logger.addHandler(size_handler)

# Rotation par temps : nouveau fichier chaque jour, garder 7 jours
time_handler = TimedRotatingFileHandler(
    'app_daily.log',
    when='midnight',
    interval=1,
    backupCount=7
)
time_handler.setFormatter(logging.Formatter('%(asctime)s - %(message)s'))
logger.addHandler(time_handler)

# Tester
for i in range(10):
    logger.info(f"Log message {i}")

## 8. Configuration avec dictConfig

Pour des configurations complexes, utilisez `dictConfig`.

In [ ]:
import logging
import logging.config

LOGGING_CONFIG = {
    'version': 1,
    'disable_existing_loggers': False,
    
    'formatters': {
        'detailed': {
            'format': '%(asctime)s | %(levelname)-8s | %(name)s | %(funcName)s:%(lineno)d | %(message)s'
        },
        'simple': {
            'format': '%(levelname)s - %(message)s'
        },
    },
    
    'handlers': {
        'console': {
            'class': 'logging.StreamHandler',
            'level': 'INFO',
            'formatter': 'simple',
            'stream': 'ext://sys.stdout',
        },
        'file': {
            'class': 'logging.handlers.RotatingFileHandler',
            'level': 'DEBUG',
            'formatter': 'detailed',
            'filename': 'app_dictconfig.log',
            'maxBytes': 10485760,  # 10MB
            'backupCount': 3,
        },
        'error_file': {
            'class': 'logging.FileHandler',
            'level': 'ERROR',
            'formatter': 'detailed',
            'filename': 'errors_dictconfig.log',
        },
    },
    
    'loggers': {
        'my_app': {
            'level': 'DEBUG',
            'handlers': ['console', 'file', 'error_file'],
            'propagate': False,
        },
        'third_party': {
            'level': 'WARNING',  # Moins verbeux pour les libs tierces
            'handlers': ['console'],
        },
    },
    
    'root': {
        'level': 'INFO',
        'handlers': ['console', 'file'],
    },
}

# Appliquer la configuration
logging.config.dictConfig(LOGGING_CONFIG)

# Utiliser
logger = logging.getLogger('my_app')
logger.debug("Debug message")
logger.info("Info message")
logger.error("Error message")

## 9. JSON Logging : Production-ready

Pour les outils modernes (Datadog, Splunk, ELK), utilisez du JSON structuré.

In [ ]:
!pip install python-json-logger -q

In [ ]:
import logging
from pythonjsonlogger import jsonlogger

# Créer un logger avec JSON formatter
logger = logging.getLogger('json_app')
logger.setLevel(logging.DEBUG)

# Handler avec JSON formatter
handler = logging.StreamHandler()
formatter = jsonlogger.JsonFormatter(
    '%(asctime)s %(levelname)s %(name)s %(funcName)s %(lineno)d %(message)s'
)
handler.setFormatter(formatter)
logger.addHandler(handler)

# Utiliser
logger.info("User logged in", extra={"user_id": 123, "ip": "192.168.1.1"})
logger.error("Database error", extra={"error_code": "DB001", "table": "users"})

### Structured Logging avec contexte

In [ ]:
import logging
import uuid
from pythonjsonlogger import jsonlogger

class StructuredLogger:
    """Logger structuré avec contexte."""
    
    def __init__(self, name):
        self.logger = logging.getLogger(name)
        self.logger.setLevel(logging.DEBUG)
        
        handler = logging.StreamHandler()
        formatter = jsonlogger.JsonFormatter(
            '%(asctime)s %(levelname)s %(name)s %(message)s'
        )
        handler.setFormatter(formatter)
        self.logger.addHandler(handler)
        
        # Contexte global
        self.context = {}
    
    def set_context(self, **kwargs):
        """Définit le contexte global."""
        self.context.update(kwargs)
    
    def _log(self, level, message, **extra):
        """Log avec contexte."""
        log_data = {**self.context, **extra}
        getattr(self.logger, level)(message, extra=log_data)
    
    def info(self, message, **extra):
        self._log('info', message, **extra)
    
    def error(self, message, **extra):
        self._log('error', message, **extra)

# Utilisation
logger = StructuredLogger('my_service')

# Définir un contexte global (ex: request ID)
request_id = str(uuid.uuid4())
logger.set_context(request_id=request_id, environment="production")

# Tous les logs incluent le contexte
logger.info("Processing request", user_id=123)
logger.info("Querying database", query="SELECT * FROM users")
logger.error("Database timeout", error_code="TIMEOUT", timeout_seconds=30)

## 10. Bonnes pratiques

### 1. Logger par module, pas en global

In [ ]:
# ❌ MAUVAIS
import logging
logging.info("Message")  # Logger root

# ✅ BON
import logging
logger = logging.getLogger(__name__)
logger.info("Message")

### 2. Ne JAMAIS logger de secrets

In [ ]:
# ❌ DANGER : Secrets dans les logs
logger.info(f"Connecting with password: {password}")
logger.debug(f"API key: {api_key}")

# ✅ BON : Masquer les secrets
logger.info("Connecting with password: ******")
logger.debug("API key: {}...{}".format(api_key[:4], api_key[-4:]))

### 3. Utiliser le bon niveau

In [ ]:
# ❌ MAUVAIS : Tout en INFO
logger.info("Variable x = 42")  # Devrait être DEBUG
logger.info("Database crashed")  # Devrait être ERROR ou CRITICAL

# ✅ BON : Niveau approprié
logger.debug("Variable x = 42")
logger.error("Database crashed", extra={"error": str(e)})

### 4. Logging paresseux (lazy logging)

In [ ]:
# ❌ MAUVAIS : Formatage inutile si DEBUG désactivé
logger.debug(f"User data: {expensive_function()}")

# ✅ BON : Lazy evaluation
logger.debug("User data: %s", expensive_function())
# expensive_function() n'est appelée que si DEBUG est activé

### 5. Ne pas logger dans des boucles intensives

In [ ]:
# ❌ MAUVAIS : 1 million de logs
for i in range(1_000_000):
    logger.debug(f"Processing item {i}")

# ✅ BON : Logger à intervalles
for i in range(1_000_000):
    if i % 10000 == 0:
        logger.info(f"Processed {i} items")

## 11. Alternative : Loguru (mention)

**Loguru** est une alternative moderne qui simplifie le logging.

In [ ]:
!pip install loguru -q

In [ ]:
from loguru import logger

# Configuration simple
logger.add("loguru_app.log", rotation="500 MB", retention="10 days")

# Utilisation
logger.info("Loguru info")
logger.debug("Loguru debug")
logger.error("Loguru error")

# Contexte automatique
logger.info("User {user} logged in", user="alice")

## Pièges courants

### 1. Logger au mauvais niveau

In [ ]:
# ❌ MAUVAIS
logger.error("Starting application")  # Ce n'est pas une erreur !
logger.info("Database connection failed")  # Devrait être ERROR

# ✅ BON
logger.info("Starting application")
logger.error("Database connection failed", exc_info=True)

### 2. Logging dans des boucles

In [ ]:
# ❌ MAUVAIS : Millions de logs
for row in df.iterrows():
    logger.debug(f"Processing row {row}")

# ✅ BON : Logs agrégés
logger.info(f"Processing {len(df)} rows")
# ... traitement ...
logger.info(f"Processed {len(df)} rows successfully")

### 3. Oublier exc_info pour les exceptions

In [ ]:
# ❌ MAUVAIS : Pas de stack trace
try:
    1 / 0
except Exception as e:
    logger.error(f"Error: {e}")

# ✅ BON : Avec stack trace
try:
    1 / 0
except Exception as e:
    logger.error("Division by zero", exc_info=True)
    # Ou : logger.exception("Division by zero")

## Mini-Exercices

### Exercice 1 : Configurer un logger JSON

Créez un logger qui :
- Écrit en JSON dans un fichier
- Inclut timestamp, niveau, message, et contexte personnalisé
- Utilise la rotation de fichiers (max 5MB, 3 backups)

In [ ]:
# Votre solution ici


### Exercice 2 : Logger pour un pipeline data

Créez un système de logging pour un pipeline de données qui :
- Loggue le début et la fin de chaque étape
- Loggue les métriques (nombre de lignes traitées, temps d'exécution)
- Loggue les erreurs avec contexte

In [ ]:
# Votre solution ici


### Exercice 3 : Masquer les données sensibles

Créez un formatter personnalisé qui masque automatiquement les mots de passe et tokens.

In [ ]:
# Votre solution ici


## Solutions

### Solution Exercice 1

In [ ]:
import logging
from logging.handlers import RotatingFileHandler
from pythonjsonlogger import jsonlogger

# Créer le logger
logger = logging.getLogger('json_exercise')
logger.setLevel(logging.DEBUG)

# Handler avec rotation
handler = RotatingFileHandler(
    'exercise1.log',
    maxBytes=5 * 1024 * 1024,  # 5MB
    backupCount=3
)

# JSON Formatter
formatter = jsonlogger.JsonFormatter(
    '%(asctime)s %(levelname)s %(name)s %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
handler.setFormatter(formatter)
logger.addHandler(handler)

# Test
logger.info("Application started", extra={"version": "1.0.0", "env": "production"})
logger.debug("Processing data", extra={"rows": 1000, "columns": 50})
logger.error("Database error", extra={"error_code": "DB001", "table": "users"})

print("Logs écrits dans exercise1.log")

### Solution Exercice 2

In [ ]:
import logging
import time
from pythonjsonlogger import jsonlogger

# Configuration
logger = logging.getLogger('data_pipeline')
logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
formatter = jsonlogger.JsonFormatter('%(asctime)s %(levelname)s %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

class DataPipeline:
    """Pipeline de données avec logging."""
    
    def __init__(self, pipeline_id):
        self.pipeline_id = pipeline_id
        self.logger = logging.getLogger(f'data_pipeline.{pipeline_id}')
    
    def run_step(self, step_name, func, *args, **kwargs):
        """Exécute une étape avec logging."""
        self.logger.info(
            f"Starting step: {step_name}",
            extra={"step": step_name, "pipeline_id": self.pipeline_id}
        )
        
        start_time = time.time()
        try:
            result = func(*args, **kwargs)
            duration = time.time() - start_time
            
            self.logger.info(
                f"Completed step: {step_name}",
                extra={
                    "step": step_name,
                    "pipeline_id": self.pipeline_id,
                    "duration_seconds": round(duration, 2),
                    "rows_processed": len(result) if hasattr(result, '__len__') else None
                }
            )
            return result
        
        except Exception as e:
            duration = time.time() - start_time
            self.logger.error(
                f"Failed step: {step_name}",
                extra={
                    "step": step_name,
                    "pipeline_id": self.pipeline_id,
                    "duration_seconds": round(duration, 2),
                    "error": str(e),
                    "error_type": type(e).__name__
                },
                exc_info=True
            )
            raise

# Utilisation
pipeline = DataPipeline("pipeline_001")

def extract():
    time.sleep(0.1)
    return [1, 2, 3, 4, 5]

def transform(data):
    time.sleep(0.1)
    return [x * 2 for x in data]

data = pipeline.run_step("extract", extract)
transformed = pipeline.run_step("transform", transform, data)

### Solution Exercice 3

In [ ]:
import logging
import re

class SensitiveDataFormatter(logging.Formatter):
    """Formatter qui masque les données sensibles."""
    
    # Patterns à masquer
    PATTERNS = [
        (re.compile(r'password["\s:=]+["\']?([^"\s]+)["\']?', re.IGNORECASE), 'password=******'),
        (re.compile(r'token["\s:=]+["\']?([^"\s]+)["\']?', re.IGNORECASE), 'token=******'),
        (re.compile(r'api[_-]?key["\s:=]+["\']?([^"\s]+)["\']?', re.IGNORECASE), 'api_key=******'),
        (re.compile(r'secret["\s:=]+["\']?([^"\s]+)["\']?', re.IGNORECASE), 'secret=******'),
    ]
    
    def format(self, record):
        """Formate le message en masquant les données sensibles."""
        # Formatter le message normalement
        original = super().format(record)
        
        # Masquer les patterns sensibles
        masked = original
        for pattern, replacement in self.PATTERNS:
            masked = pattern.sub(replacement, masked)
        
        return masked

# Configuration
logger = logging.getLogger('secure_app')
logger.setLevel(logging.INFO)

handler = logging.StreamHandler()
formatter = SensitiveDataFormatter('%(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)

# Test
logger.info("User login with password=super_secret_123")
logger.info("API call with api_key=sk_live_abcdefghijklmnop")
logger.info("Token: Bearer xyz789abc")
logger.info("Normal message without secrets")